In [ ]:
from sagemaker import get_execution_role
import os


role = get_execution_role()

In [ ]:
from src.generated.shapes import ContainerDefinition, ProductionVariant
# Endpoint Invoking 
from src.generated.resources import Model, EndpointConfig, Endpoint, TrainingJob
from time import gmtime, strftime


os.environ[
    'SAGEMAKER_ADMIN_CONFIG_OVERRIDE'] = '/Users/nargokul/workspace/sagemaker-code-gen/sample/sagemaker/2017-07-24/default-configs-inference.json'

key = f'xgboost-iris-{strftime("%H-%M-%S", gmtime())}'
print("key", key)

model = Model.create(
    model_name=key,
    primary_container=ContainerDefinition(
        image='246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.7-1',
        model_data_url='s3://sagemaker-us-west-2-211125564141/DEMO-scikit-iris/output/xgboost-iris-2024-05-17-02-10-05/output/serve.tar.gz',
        # here we are getting model data from the training job 
        environment={
            'LOCAL_PYTHON': '3.10.12',
            'MODEL_CLASS_NAME': 'xgboost.sklearn.XGBClassifier',
            'SAGEMAKER_CONTAINER_LOG_LEVEL': '10',
            'SAGEMAKER_PROGRAM': 'inference.py',
            'SAGEMAKER_REGION': 'us-west-2',
            'SAGEMAKER_SERVE_SECRET_KEY': '3a459322560a181436866602ddfbb7c16ea97046e92845de43a5ac80f7604451',
            'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'
        }
    ),
    execution_role_arn=role,
)


'''
if the model is already created then 
we can use Model.get() to retrive the model
'''

endpoint_config = EndpointConfig.create(
    endpoint_config_name=key,
    production_variants=[
        ProductionVariant(
            variant_name=key,
            initial_instance_count=1,
            instance_type='ml.m5.xlarge',
            model_name=key
        )
    ]
)

endpoint: Endpoint = Endpoint.create(
    endpoint_name=key,
    endpoint_config_name=key 
)

In [ ]:
from src.generated.resources import Endpoint
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sagemaker.base_serializers import NumpySerializer

key = 'xgboost-iris-5-07'
endpoint = Endpoint.get(endpoint_name=key)
import io
import numpy as np



dataset = loadtxt('data/pima-indians-diabetes.data.csv', delimiter=",")
# split data into X and y
X = dataset[:, 0:8]
Y = dataset[:, 8]
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
serializer = NumpySerializer()


def deserialise(response):
    return np.load(io.BytesIO(response['Body'].read()))


invoke_result = endpoint.invoke(body=serializer.serialize(X_test),
                                content_type='application/x-npy',
                                accept='application/x-npy')

print("Endpoint Response:", deserialise(invoke_result))

In [ ]:
from src.generated.resources import Endpoint
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sagemaker.base_serializers import NumpySerializer

endpoint = Endpoint.get(endpoint_name=key)
dataset = loadtxt('data/pima-indians-diabetes.data.csv', delimiter=",")

# split data into X and y
X = dataset[:, 0:8]
Y = dataset[:, 8]
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
serializer = NumpySerializer()


def deserialise(response):
    return [
        res_part
        for res_part in response['Body']
    ]


invoke_result = endpoint.invoke_with_response_stream(body=serializer.serialize(X_test),
                                                     content_type='application/x-npy',
                                                     accept='application/x-npy')

print("Endpoint Response:", deserialise(invoke_result))